In [ ]:
import pandas as pd
import numpy as np
!pip install plotly
pd.options.plotting.backend = "plotly"
import plotly.io as pio
import plotly.express as px
pio.templates.default = "plotly_white"

In [ ]:
# Testing Plotly, Looks sick doesn't it!??
TestDF=pd.DataFrame(np.random.randn(100,5),index=pd.date_range('1/1/15',periods=100),
                columns=['IBM','MSFT','GOOG','VERZ','APPL'])
TestDF.cumsum().plot()

Let's import the CSVs

In [ ]:
sample_submission = pd.read_csv("/datasets/real-or-not-dataset/sample_submission.csv")
test = pd.read_csv("/datasets/real-or-not-dataset/test.csv")
train = pd.read_csv("/datasets/real-or-not-dataset/train.csv")

Let's do just look at the train dataset first

In [ ]:
print(train.target.describe())
train.head(len(train))

count    7613.00000
mean        0.42966
std         0.49506
min         0.00000
25%         0.00000
50%         0.00000
75%         1.00000
max         1.00000
Name: target, dtype: float64


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


Looks like the data is pretty evenly split between real and not real

In [ ]:
false_true_counts = train.target.value_counts()
false_true_counts = false_true_counts.to_frame()
false_true_counts['indx'] = false_true_counts.index
false_true_counts.columns=['counts','target']
false_true_counts.plot(x ='target', y='counts', kind = 'bar', title='Distibution of Real and Fake tweets')

In [ ]:
nan_rows = train[train['location'].isnull()]
nan_rows

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


So we have 33% of the rows don't have a location

In [ ]:
is_notreal = nan_rows[nan_rows['target']==0].shape[0] #1 is real
notReal_noloc = is_notreal/nan_rows.shape[0]
print("Real tweets without keyword: {0:0.2f}%".format((notReal_noloc*100)))

Real tweets without keyword: 57.56%


A slightly larger number of rows that don't have a location are not real

This might just be a coincidence thing though

In [ ]:
nan_rows.count()

id          2533
keyword     2472
location       0
text        2533
target      2533
dtype: int64

In [ ]:
nan_keywords = train[train['keyword'].isnull()]
print("Total tweets without keyword: {}".format(nan_keywords.shape[0]))

Total tweets without keyword: 61


In [ ]:
real_nokeyword = (nan_keywords[nan_keywords['target']==1].shape[0]/nan_keywords.shape[0])
print("Real tweets without keyword: {0:0.2f}%".format((real_nokeyword*100)))

Real tweets without keyword: 68.85%


Weirdly enough, the if the tweet doesn't have a keyword, it has a higher change of being real

Interesting huhh?

In [ ]:
locations = train
locations[locations.location.notnull()]

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
36,54,ablaze,Pretoria,@PhDSquares #mufc they've built so much hype a...,0
37,55,ablaze,World Wide!!,INEC Office in Abia Set Ablaze - http://t.co/3...,1
39,57,ablaze,Paranaque City,Ablaze for you Lord :D,0
40,59,ablaze,Live On Webcam,Check these out: http://t.co/rOI2NSmEJJ http:/...,0
42,62,ablaze,milky way,Had an awesome time visiting the CFC head offi...,0


In [ ]:
!pip install flashgeotext
# for finding the city names properly
from flashgeotext.geotext import GeoText
geotext = GeoText(use_demo_data=True)

from collections import Counter

true_locations = train[train['target']==1]
false_locations = train[train['target']==0]

2020-06-24 18:38:54.569 | DEBUG    | flashgeotext.lookup:add:194 - cities added to pool
2020-06-24 18:38:54.574 | DEBUG    | flashgeotext.lookup:add:194 - countries added to pool
2020-06-24 18:38:54.575 | DEBUG    | flashgeotext.lookup:_add_demo_data:225 - demo data loaded for: ['cities', 'countries']


In [ ]:
all_true_locations = true_locations.location.str.cat(sep=' ')#convert to string
shittygeo_dict_1 = geotext.extract(input_text=all_true_locations, span_info=False)
temp_dict_1 = {}
for key in shittygeo_dict_1['countries']:
    temp_dict_1.update({str(key) : int(shittygeo_dict_1['countries'][key]['count'])})
count_true_country_df = pd.DataFrame(temp_dict_1.items(), columns=['Country', 'Occurrences'])

In [ ]:
all_false_locations = false_locations.location.str.cat(sep=' ')#convert to string
shittygeo_dict_2 = geotext.extract(input_text=all_false_locations, span_info=False)
temp_dict_2 = {}
for key in shittygeo_dict_2['countries']:
    temp_dict_2.update({str(key) : int(shittygeo_dict_2['countries'][key]['count'])})
count_false_country_df = pd.DataFrame(temp_dict_2.items(), columns=['Country', 'Occurrences'])

As plotly is dumb and cannot only show the top ones, so we converted those counters to a dataframe in pandas, so now we can sort by top and plot it

In [ ]:
count_true_country_df.sort_values(['Occurrences'],ascending=False, inplace = True)
count_true_country_df[:8].plot(x='Country', y='Occurrences', kind='bar', title ="Top Counties with real tweets")

In [ ]:
count_false_country_df.sort_values(['Occurrences'], ascending=False, inplace = True)
count_false_country_df[:8].plot(x='Country', y='Occurrences', kind='bar', title="Top Counties with non-real tweets")

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
from nltk.stem.snowball import SnowballStemmer

In [ ]:
train.keyword[train['keyword'].isnull()] = ''
train.location[train['location'].isnull()] = ''

In [ ]:
keyword_string = train.keyword.str.cat(sep=' ')
tokenized_word = word_tokenize(keyword_string)
stemmer = SnowballStemmer("english", ignore_stopwords=True)
clean_tokenized_word = [str(stemmer.stem(word)) for word in tokenized_word]
clean_tokenized_word = [w for w in tokenized_word if not w in stopwords]
# filtered_keyword=[]
# for w in tokenized_word:
#     if w not in stopwords:
#         filtered_keyword.append(w)
fdist = nltk.FreqDist(clean_tokenized_word)
fd = pd.DataFrame(fdist.most_common(30),columns = ["Word","Frequency"]).drop([0]).reindex()

In [ ]:
px.bar(fd, x='Word', y='Frequency', title='Frequency of top Keywords')

In [ ]:
train.head()

,id,keyword,location,text,target
0,,,,,
1,,,,,
2,,,,,
3,,,,,
4,,,,,
